## Generating a Fantasy Premier League starting team via Monte Carlo Simulation

##### Fantasy Premier League (FPL) is The official fantasy game of the English premier league. Players act as managers running virtual teams, built within budgetary constraints, with the goal of accumulating as many points as possible based on the real-life performances of their selected players. The game runs throughout the football season, with points awarded weekly according to player performance.

##### A lot of effort is usually (and rightly) put into modelling player and team performance throughout the season in order to make the right decisions in terms of player transfers, captaincy choices and chip usage. However, football is extremely dynamic, especially from season to season, and thus the first few months of a season can be highly volatile. Teams change because of transfers and managerial changes and players do too. Because of this, trying to model player performance in the first few months of the season is, in my opinion, extremely difficult. New highly-rated players join the league and struggle. Previously unremarkable players, both new and seasoned, start a new season and flourish. Highly-regarded seasoned players hit slumps and have poor seasons, and new young players that few managers had in the plans have breakout seasons. 

##### The most common strategy, in my experience, is to go with the "proven" players at the start of the season, (with maybe 2/3 spots reserved for 'gut feeling' players) and adjusting the team as the season progresses. However, the challenge is usually trying to find the best combination of players within the starting budget constraint of 100 million. Also, the budget is not the only constraint imposed on team construction. There are 'single-team' constraints (no more than 3 players from the same team) and positional constraints ( exactly 2 goalkeepers, 5 defenders, 5 midfielders and 3 forwards). Since I usually select the "proven" players through a combination of quick google searches, simple arithmetic calculations, trial and error and gut feeling, I thought it would be helpful to try and find a solution to the optimization problem (i.e. the best combination of 15 players based on the previous 3 seasons while obeying all constraints). Since the actual solved solution of the problem is complicated, I chose to use the Monte Carlo method; simulating many teams (10,000 iterations in this case) and identifying the best teams in terms of average performance over the last three seasons in each case. I then evaluated the performance of the teams through the first 10 games of the season (which I think is a reasonable timeframe after which enough time has passed to try and model season-specific player patterns) to determine whether the solution is actually helpful.

In [1]:
# libraries
import pandas as pd

# dataframe from excel file with fpl player performance data from the 2016-17 to 2023-24 seasons
df = pd.read_csv("cleaned_merged_seasons_team_aggregated.csv")

C:\Users\tito\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\tito\AppData\Local\Temp\ipykernel_16096\1220514799.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cleaned_merged_seasons_team_aggregated.csv")


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126076 entries, 0 to 126075
Data columns (total 41 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   season_x             126076 non-null  object 
 1   name                 126076 non-null  object 
 2   position             126076 non-null  object 
 3   team_x               106042 non-null  object 
 4   assists              126076 non-null  int64  
 5   bonus                126076 non-null  int64  
 6   bps                  126076 non-null  int64  
 7   clean_sheets         126076 non-null  int64  
 8   creativity           126076 non-null  float64
 9   element              126076 non-null  int64  
 10  fixture              126076 non-null  int64  
 11  goals_conceded       126076 non-null  int64  
 12  goals_scored         126076 non-null  int64  
 13  ict_index            126076 non-null  float64
 14  influence            126076 non-null  float64
 15  kickoff_time     

In [3]:
df.head()

,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,...,transfers_in,transfers_out,value,was_home,yellow_cards,GW,points,team_goals_scored,team_goals_conceded,team_goals_diff
0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,...,0,0,55,False,0,1,NaN,NaN,NaN,NaN
1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,142,...,0,0,60,True,0,1,NaN,NaN,NaN,NaN
2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,16,...,0,0,80,True,0,1,NaN,NaN,NaN,NaN
3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,...,0,0,50,False,0,1,NaN,NaN,NaN,NaN
4,2016-17,Abdul Rahman Baba,DEF,NaN,0,0,0,0,0.0,80,...,0,0,55,True,0,1,NaN,NaN,NaN,NaN


In [4]:
# loading 24/25 data
df_24_25 = pd.DataFrame()
for i in range (1,39):
    data_i = pd.read_csv(f'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2024-25/gws/gw{i}.csv')
    df_24_25 = pd.concat([df_24_25, data_i], ignore_index=True)

In [5]:
df_24_25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        27605 non-null  object 
 1   position                    27605 non-null  object 
 2   team                        27605 non-null  object 
 3   xP                          27605 non-null  float64
 4   assists                     27605 non-null  int64  
 5   bonus                       27605 non-null  int64  
 6   bps                         27605 non-null  int64  
 7   clean_sheets                27605 non-null  int64  
 8   creativity                  27605 non-null  float64
 9   element                     27605 non-null  int64  
 10  expected_assists            27605 non-null  float64
 11  expected_goal_involvements  27605 non-null  float64
 12  expected_goals              27605 non-null  float64
 13  expected_goals_conceded     276

In [6]:
# streamlining data for concatenation
df_24_25['season_x'] = "2024-25"
df_24_25['team_x'] = df_24_25['team']

In [8]:
# contenating 24/25 data to main dataframe
df = pd.concat([df, df_24_25],join = "inner", ignore_index=True)

# data cleaning
df = df.replace({'GKP': 'GK'})

indices_to_drop = df[df['position'] == 'AM'].index
df= df.drop(indices_to_drop)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153359 entries, 0 to 153680
Data columns (total 35 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   season_x           153359 non-null  object 
 1   name               153359 non-null  object 
 2   position           153359 non-null  object 
 3   team_x             133325 non-null  object 
 4   assists            153359 non-null  int64  
 5   bonus              153359 non-null  int64  
 6   bps                153359 non-null  int64  
 7   clean_sheets       153359 non-null  int64  
 8   creativity         153359 non-null  float64
 9   element            153359 non-null  int64  
 10  fixture            153359 non-null  int64  
 11  goals_conceded     153359 non-null  int64  
 12  goals_scored       153359 non-null  int64  
 13  ict_index          153359 non-null  float64
 14  influence          153359 non-null  float64
 15  kickoff_time       153359 non-null  object 
 16  minutes

In [10]:
# data into seasons
df_20_21 = df[df['season_x'] == "2020-21"]
df_21_22 = df[df['season_x'] == "2021-22"]
df_22_23 = df[df['season_x'] == "2022-23"]
df_23_24 = df[df['season_x'] == "2023-24"]
df_24_25 = df[df['season_x'] == "2024-25"]

dataframe_dict = {
    "2020-21" : df_20_21,
    "2021-22" : df_21_22,
    "2022-23" : df_22_23,
    "2023-24" :df_23_24,
    "2024-25" : df_24_25,
}


In [11]:
# calculating player averages from the previous three seasons for each season
season_dict = {
    "2020-21" : ["2017-18", "2018-19", "2019-20"],
    "2021-22" : ["2018-19", "2019-20", "2020-21"],
    "2022-23" : ["2019-20", "2020-21", "2021-22"],
    "2023-24" : ["2020-21", "2021-22", "2022-23"],
    "2024-25" : ["2021-22", "2022-23", "2023-24"],
    "2025-26" : ["2022-23", "2023-24", "2024-25"]
}
all_season_data = pd.DataFrame()
for key, value in season_dict.items():
    hist_data = df[df['season_x'].isin(value)]
    merged_data = hist_data.groupby('name').agg(
        mean=('total_points', 'mean'),
        variance=('total_points', 'var'),
        total_minutes=('minutes', 'sum'),
        games_played=('minutes', lambda x: (x > 0).sum())
    ).reset_index()
    
    merged_data['avg_minutes_per_game'] = merged_data['total_minutes'] / merged_data['games_played']
    merged_data = merged_data[(merged_data['avg_minutes_per_game'] > 10) & (merged_data['games_played'] > 30)]


    current_season_df = df[df['season_x'] == key]
    rel_players_df = current_season_df[current_season_df['name'].isin(merged_data['name'])]
    current_summary = rel_players_df.groupby('name')['total_points'].sum().reset_index(name = 'total points')
    final_data = pd.merge(merged_data,current_summary, on='name')
    
    player_info = rel_players_df[['name', 'value', 'position', 'team_x']].drop_duplicates(subset='name')
    final_data = pd.merge(final_data, player_info, on='name', how='left')

    final_data['season'] = key
    all_season_data = pd.concat([all_season_data,final_data], ignore_index=True)


In [12]:
all_season_data

,name,mean,variance,total_minutes,games_played,avg_minutes_per_game,total points,value,position,team_x,season
0,Aaron Cresswell,3.105263,8.637269,3069,36,85.250000,153,50,DEF,West Ham,2020-21
1,Aaron Mooy,2.973684,7.918208,3066,36,85.166667,0,50,MID,Brighton,2020-21
2,Abdoulaye Doucouré,3.578947,9.007112,3324,37,89.837838,80,55,MID,Everton,2020-21
3,Alex Oxlade-Chamberlain,2.473684,7.283073,1718,35,49.085714,21,65,MID,Liverpool,2020-21
4,Alexandre Lacazette,3.631579,12.022760,2197,32,68.656250,129,85,FWD,Arsenal,2020-21
...,...,...,...,...,...,...,...,...,...,...,...
1002,Willy Boly,0.982456,4.796150,3021,41,73.682927,6,45,DEF,Nott'm Forest,2024-25
1003,Wout Faes,2.363636,8.176136,2786,31,89.870968,55,40,DEF,Leicester,2024-25
1004,Yoane Wissa,2.929825,9.941934,5343,102,52.382353,185,60,FWD,Brentford,2024-25
1005,Youri Tielemans,2.482456,7.579336,6583,95,69.294737,121,55,MID,Aston Villa,2024-25


In [13]:
# brief look at the player means, variances, games and minutes played compared to their performance during the current season
q = all_season_data[all_season_data['season'] =="2024-25"]
q.sort_values(['mean', 'variance'], ascending=False).head(10)

,name,mean,variance,total_minutes,games_played,avg_minutes_per_game,total points,value,position,team_x,season
796,Erling Haaland,6.434211,32.942281,5320,66,80.606061,181,150,FWD,Man City,2024-25
922,Mohamed Salah,6.271930,27.916550,8579,105,81.704762,344,125,MID,Liverpool,2024-25
744,Bukayo Saka,5.324561,16.893728,9083,111,81.828829,127,100,MID,Arsenal,2024-25
976,Son Heung-min,4.802632,20.133860,5820,71,81.971831,129,100,MID,Spurs,2024-25
944,Ollie Watkins,4.684211,17.191430,9301,109,85.330275,186,90,FWD,Aston Villa,2024-25
905,Martin Ødegaard,4.681416,16.129741,9012,108,83.444444,116,85,MID,Arsenal,2024-25
843,Jarrod Bowen,4.675439,18.203462,9239,108,85.546296,193,75,MID,West Ham,2024-25
741,Bruno Borges Fernandes,4.500000,15.693333,6435,72,89.375000,174,85,MID,Man Utd,2024-25
953,Phil Foden,4.464912,23.171324,6817,95,71.757895,105,95,MID,Man City,2024-25
732,Benjamin White,4.447368,13.210526,6041,75,80.546667,50,65,DEF,Arsenal,2024-25


In [ ]:
# libraries
import numpy as np

# Function for sampling an FPL team whilst obeying team and positional constraints
def team_generator(season):
    team = []
    data = all_season_data[all_season_data['season'] == season]
    team_count_dict = {k : 0 for k in data['team_x'].unique().tolist()}
    positions = {
        'GK': 2,
        'DEF': 5,
        'MID': 5,
        'FWD': 3
    }

    for key, value in positions.items():
        position_data = data[data['position'] == key]
        while value > 0:
            chosen_player = position_data.sample(1, random_state = np.random.randint(0, 1e6))
            player_team = chosen_player['team_x'].tolist()[0]
            team_count_dict[player_team] +=1
            if team_count_dict[player_team] >= 3:
                continue
            else:
                team.extend(chosen_player['name'])
            value-=1
    
    return team


In [ ]:
# Function for running the simulation and tracking the best sampled teams
def team_simulator(iterations, season):
    i = 0
    results_df = pd.DataFrame(columns=['team_value', 'team_mean', 'team_variance', 'team_players'])

    while i != iterations:
        gen_team = team_generator(season)

        season_data = all_season_data[all_season_data['season'] == season]
        team_df = season_data[season_data['name'].isin(gen_team)]

        team_value = team_df['value'].sum()

        if team_value < 850 or team_value > 1000:
            continue
        else:
            row_data = {
                'team_players': gen_team,
                'team_value': team_value,
                'team_mean': team_df['mean'].sum(),
                'team_variance': team_df['variance'].sum()
            }
            row_df = pd.DataFrame([row_data])
            results_df = pd.concat([results_df, row_df], ignore_index=True)

            i += 1
            print(i)
    results_df = results_df.sort_values('team_mean', ascending=False)
    return results_df


In [ ]:
# simulating the 24-25 season
np.random.seed(123) 
res = team_simulator(10000, "2024-25")

In [ ]:
# top 30 teams from the 2024-25 simulation
res.head(30)

In [ ]:
res['team_mean'].describe()

In [ ]:
# identifying the "best" team in terms of average points
best_team = res.loc[7140,"team_players"]

In [ ]:
best_team

In [ ]:
# summary of the best team
q = all_season_data[all_season_data['season'] =="2024-25"]
c =q[q['name'].isin(best_team)]

print(f"total points team acquired during the 2024-25 season: {c['total points'].sum()}")
c.sort_values(['position','mean'],ascending=False)

In [ ]:
# function for assigning players "starting" or "bench" and captaincy status depending on previous performance
def starting_team_selector(team, season):
    df  = all_season_data[all_season_data['season'] ==season]
    team_data = df[df['name'].isin(team)]
    team_data = team_data.sort_values(['position','mean'],ascending=False)
    formations = [[1,3,4,3],[1,3,5,2],[1,4,3,3],[1,4,4,2],[1,4,5,1],[1,5,3,2],[1,5,4,1]]
    positions = ['GK','DEF','MID','FWD']

    best_points = 0
    best_formation = [0,0,0,0]
    best_team = [ ]

    for formation in formations:
        starting_team = []
        for n, pos in zip(formation,positions):
            pos_data = team_data[team_data['position'] == pos]
            pos_starters = pos_data.head(n)
            starting_team.extend(pos_starters['name'])
        current_points = team_data[team_data['name'].isin(starting_team)]['mean'].sum()
        if current_points > best_points:
            best_points = current_points
            best_formation = formation
            best_team = starting_team

    team_data['starting_status'] = team_data['name'].apply(lambda x: 'start' if x in best_team else 'bench')
    team_data['captaincy'] = 0
    idx_max = team_data['mean'].idxmax()
    team_data.loc[idx_max, 'captaincy'] = 1
    
    return team_data

In [ ]:
starting_team_selector(best_team, "2024-25")

In [ ]:
# function for evaluating team perfomance over the firt n games of the season
def team_evaluater(n, season, team):
    best_team = starting_team_selector(team, season)
    best_team.index = best_team['name']
    team_dict = best_team.apply(lambda row: [row['starting_status'], row['captaincy']], axis=1).to_dict()
    df =  dataframe_dict[season]
    total_points = []

    for round in range(1, n+1):
        gw_points = 0
        data = df[df['round'] == round]
        for key, value in team_dict.items():
            if value[0] == 'start':
                player_gw_data = data[data['name'] == key]
                try:
                    if value[1] == 1:
                        gw_points += 2 * int(player_gw_data['total_points'].sum())
                    else:
                        gw_points += int(player_gw_data['total_points'].sum())
                except:
                    gw_points +=0
        total_points.append(gw_points)
        
    return team_dict, total_points


In [ ]:
team_evaluater(n=10, season="2024-25",team= best_team)

In [ ]:
# evaluating the top 10 teams sampled for the 2024-25 season
top_teams_24_25 = res.head(10)
top_team_dict = top_teams_24_25.apply(lambda row: [row['team_mean'], row['team_value'], row['team_variance'], row['team_players']], axis=1).to_dict()
summary_df = pd.DataFrame()

for key, value in top_team_dict.items():
     starters, points_list = team_evaluater(n=10, season="2024-25",team= value[3])
     point_dict = {index: value for index, value in enumerate(points_list, start=1)}
     sum_dict = {
          'team_id': key,
          'team_mean' : value[0],
          'team_value' : value[1],
          'team_variance': value[2],
          'team_info' : starters}
     total = {'total' : sum(points_list)}
     sum_dict.update(point_dict)
     sum_dict.update(total)
     new_row = pd.DataFrame([sum_dict])
     summary_df = pd.concat([summary_df, new_row], axis=0)
     
summary_df.reset_index(inplace=True)
summary_df

In [ ]:
# evaluating the top 10 teams sampled for the 2023-24 season
np.random.seed(123) 
res_21_22 = team_simulator(10000, "2023-24")


top_teams_21_22 = res_21_22.head(10)
top_team_dict = top_teams_21_22.apply(lambda row: [row['team_mean'], row['team_value'], row['team_variance'], row['team_players']], axis=1).to_dict()
summary_df = pd.DataFrame()

for key, value in top_team_dict.items():
     starters, points_list = team_evaluater(n=10, season="2021-22",team= value[3])
     point_dict = {index: value for index, value in enumerate(points_list, start=1)}
     sum_dict = {
          'team_id': key,
          'team_mean' : value[0],
          'team_value' : value[1],
          'team_variance': value[2],
          'team_info' : starters}
     total = {'total' : sum(points_list)}
     sum_dict.update(point_dict)
     sum_dict.update(total)
     new_row = pd.DataFrame([sum_dict])
     summary_df = pd.concat([summary_df, new_row], axis=0)
     
summary_df.reset_index(inplace=True)
summary_df

In [74]:
# 25/26 predictions
import numpy as np
from rapidfuzz import fuzz, process

season_dict = {
    "2025-26" : ["2022-23", "2023-24", "2024-25"]
}
for key, value in season_dict.items():
    hist_data = df[df['season_x'].isin(value)]
    merged_data = hist_data.groupby('name').agg(
        mean=('total_points', 'mean'),
        variance=('total_points', 'var'),
        total_minutes=('minutes', 'sum'),
        games_played=('minutes', lambda x: (x > 0).sum())
    ).reset_index()
    
    merged_data['avg_minutes_per_game'] = merged_data['total_minutes'] / merged_data['games_played']
    merged_data = merged_data[(merged_data['avg_minutes_per_game'] > 10) & (merged_data['games_played'] > 30)]

    
new_season_data = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2025-26/cleaned_players.csv')
new_season_data['full_name'] = new_season_data['first_name'] + " " + new_season_data['second_name']

# Function to match names from df1 to df2
def match_name(name, choices, scorer=fuzz.token_sort_ratio, threshold=75):
    match = process.extractOne(name, choices, scorer=scorer)
    if match and match[1] >= threshold:
        return match[0]
    return None

# Match names
merged_data['matched_name'] = merged_data['name'].apply(lambda x: match_name(x, new_season_data['full_name']))

# Merge using matched names
merged_data = merged_data.merge(new_season_data, left_on='matched_name', right_on='full_name', suffixes=('_df1', '_df2'))
merged_data = merged_data.rename({'element_type':'position','now_cost': 'value'}, axis=1)

def team_generator(season):
    team = []
    data = all_season_data[all_season_data['season'] == season]
    team_count_dict = {k : 0 for k in data['team_x'].unique().tolist()}
    positions = {
        'GK': 2,
        'DEF': 5,
        'MID': 5,
        'FWD': 3
    }

    for key, value in positions.items():
        position_data = data[data['position'] == key]
        while value > 0:
            chosen_player = position_data.sample(1, random_state = np.random.randint(0, 1e6))
            player_team = chosen_player['team_x'].tolist()[0]
            team_count_dict[player_team] +=1
            if team_count_dict[player_team] >= 3:
                continue
            else:
                team.extend(chosen_player['name'])
            value-=1
    
    return team

def team_simulator(iterations, season):
    i = 0
    results_df = pd.DataFrame(columns=['team_value', 'team_mean', 'team_variance', 'team_players'])

    while i != iterations:
        gen_team = team_generator(season)

        season_data = merged_data
        team_df = season_data[season_data['name'].isin(gen_team)]

        team_value = team_df['value'].sum()
        team_len = len(team_df)
        if team_len != 15:
            continue

        if team_value < 850 or team_value > 1000:
            continue
        else:
            row_data = {
                'team_players': gen_team,
                'team_value': team_value,
                'team_mean': team_df['mean'].sum(),
                'team_variance': team_df['variance'].sum()
            }
            row_df = pd.DataFrame([row_data])
            results_df = pd.concat([results_df, row_df], ignore_index=True)

            i += 1
            print(i)
    results_df = results_df.sort_values('team_mean', ascending=False)
    return results_df
simmed_25_26 = team_simulator(1000, "2024-25")

C:\Users\tito\AppData\Local\Temp\ipykernel_16096\1499109146.py:88: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, row_df], ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [75]:
pd.set_option('display.max_colwidth', None)
simmed_25_26.head(15)

,team_value,team_mean,team_variance,team_players
396,1000,48.118663,189.800289,"[André Onana, Aaron Ramsdale, Marcos Senesi, Dan Burn, James Tarkowski, Tosin Adarabioyo, Jarrad Branthwaite, Sander Berge, Dango Ouattara, Bruno Borges Fernandes, Phil Foden, Leandro Trossard, Ollie Watkins, Yoane Wissa, Erling Haaland]"
517,980,46.380366,180.454432,"[José Malheiro de Sá, Alisson Ramses Becker, Trevoh Chalobah, Nathan Aké, Lewis Dunk, Benjamin White, Illia Zabarnyi, Martin Ødegaard, Mohamed Salah, Crysencio Summerville, Edson Álvarez Velázquez, Phil Foden, Nicolas Jackson, Chris Wood, Yoane Wissa]"
614,955,46.266314,185.858812,"[José Malheiro de Sá, Jordan Pickford, Murillo Santiago Costa dos Santos, Joël Veltman, Kenny Tete, Tyrick Mitchell, Lewis Dunk, Alexis Mac Allister, Cole Palmer, Martin Ødegaard, James Maddison, Ilkay Gündogan, Jean-Philippe Mateta, Chris Wood, Matheus Santos Carneiro Da Cunha]"
702,995,45.948694,183.229757,"[Bernd Leno, Alisson Ramses Becker, Tyrick Mitchell, Jarrad Branthwaite, Antonee Robinson, Jan Paul van Hecke, Tosin Adarabioyo, Bryan Mbeumo, James Ward-Prowse, Declan Rice, Lucas Tolentino Coelho de Lima, Cole Palmer, Matheus Santos Carneiro Da Cunha, Evan Ferguson, Erling Haaland]"
851,945,45.822771,150.587734,"[Ederson Santana de Moraes, David Raya Martin, Mads Roerslev Rasmussen, Vitalii Mykolenko, Tyrick Mitchell, Adam Smith, Fabian Schär, Alexis Mac Allister, Andreas Hoelgebaum Pereira, Moisés Caicedo Corozo, Facundo Buonanotte, Hwang Hee-chan, Erling Haaland, Yoane Wissa, Ollie Watkins]"
91,985,45.380952,187.460750,"[André Onana, Jason Steele, Pervis Estupiñán, James Tarkowski, William Saliba, Ethan Pinnock, Aaron Wan-Bissaka, Cole Palmer, Mohamed Salah, Bukayo Saka, Harrison Reed, Vitaly Janelt, Taiwo Awoniyi, Nicolas Jackson, Chris Wood]"
136,920,45.072655,159.607688,"[Alisson Ramses Becker, André Onana, James Tarkowski, Nathan Collins, Virgil van Dijk, Fabian Schär, William Saliba, Carlos Henrique Casimiro, Harvey Barnes, Matheus Luiz Nunes, Bernardo Veiga de Carvalho e Silva, Adama Traoré, Ollie Watkins, Jean-Philippe Mateta, Chris Wood]"
805,990,44.314944,157.720074,"[Emiliano Martínez Romero, David Raya Martin, James Tarkowski, Trevoh Chalobah, Chris Richards, Konstantinos Tsimikas, Antonee Robinson, Mikkel Damsgaard, Moisés Caicedo Corozo, Jefferson Lerma Solís, Harvey Elliott, Bryan Mbeumo, Erling Haaland, Alexander Isak, Ollie Watkins]"
589,985,44.281915,169.483352,"[José Malheiro de Sá, André Onana, Kieran Tierney, Marc Guéhi, Matt Doherty, Antonee Robinson, Tosin Adarabioyo, Phil Foden, Mohamed Salah, Leandro Trossard, Mikkel Damsgaard, Dejan Kulusevski, Taiwo Awoniyi, Alexander Isak, Cody Gakpo]"
430,935,44.271930,159.609992,"[André Onana, David Raya Martin, Antonee Robinson, Diogo Dalot Teixeira, Ibrahima Konaté, Benjamin White, Lewis Dunk, James Maddison, Harvey Barnes, James Garner, Crysencio Summerville, Dwight McNeil, Yoane Wissa, Darwin Núñez Ribeiro, Erling Haaland]"
